## Zuber

##### Initialization

In [ ]:
# Import libraries that might be necessary

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm
from scipy.interpolate import UnivariateSpline
import scipy.stats as stats

# Import data
comp_count_trip = 'https://raw.githubusercontent.com/DHE42/zuber/refs/heads/main/moved_project_sql_result_01.csv'
comp_count_trip_df = pd.read_csv(comp_count_trip)

dropoff_avg_trip = 'https://raw.githubusercontent.com/DHE42/zuber/refs/heads/main/moved_project_sql_result_04.csv'
dropoff_avg_trip_df = pd.read_csv(dropoff_avg_trip)






Is this python?
